In [37]:
import numpy as np
import pulp
import math
import copy
from scipy.stats import weibull_min
from scipy.special import gamma



In [38]:
LOWER_BOUND_transport = 1
UPPER_BOUND_transport = 20

LOWER_BOUND_demand = 100
UPPER_BOUND_demand = 400

LOWER_BOUND_production = 20
UPPER_BOUND_production = 50

LOWER_BOUND_service = 5
UPPER_BOUND_service = 15

LOWER_BOUND_extra_charge = 20
UPPER_BOUND_extra_charge = 40

LOWER_BOUND_safety_stock = 0
UPPER_BOUND_safety_stock = 100

LOWER_BOUND_service_add = 5
UPPER_BOUND_service_add = 30



In [39]:
def generate_data(n, m, verbose=True):

    d_k = np.random.randint(LOWER_BOUND_demand, UPPER_BOUND_demand,  size = n) #спрос в k узле
    D = np.sum(d_k) # весь спрос

#генерация вместимостей для РЦ 
    mu = np.zeros(2, dtype=int)
    mu[0] = np.random.randint(int(0.3*D), int(0.7*D))
    mu[1] = D - mu[0]
    #генерация возможных расширений пропускных способностей
    mu_add = np.zeros(2, dtype=int)
    for j in range(2):
        mu_add[j] = np.random.choice([0, int(0.1*mu[j]), int(0.3*mu[j]), int(0.5*mu[j]), int(0.7*mu[j]), int(mu[j]), int(1.2*mu[j])])
        

#генерация мощностей производственных центров
    nu = np.zeros(m, dtype=int)
    remaining = D
    #распределение между ПЦ
    for i in range(m - 1):
        min_val = max(1, int(0.3 * remaining))
        max_val = min(int(0.7 * remaining), remaining - (m - i - 1))
        nu[i] = np.random.randint(min_val, max_val + 1)
        remaining -= nu[i]
    nu[-1] = remaining  #последний ПЦ получает остаток
    #генерация возможных повышений мощностей
    nu_add = np.array([
        np.random.choice([0, int(0.1 * nu_i), int( nu_i)])
        for nu_i in nu
    ])

    Cp_i = np.random.randint(LOWER_BOUND_production, UPPER_BOUND_production,  size = m) #затраты на производство единицы продукции для i производственного центра
    Cd_j = np.random.randint(LOWER_BOUND_service, UPPER_BOUND_service, size = 2) #затраты на единицу продукции для j распределительного центра
    Cd_j_add = np.zeros(2, dtype=int)
    for j in range(2):
        Cd_j_add[j] = np.random.choice([0, np.random.randint(LOWER_BOUND_service, UPPER_BOUND_service)])
    Cp_i_add = np.zeros(m, dtype=int)
    for i in range(m):
        Cp_i_add[i] = np.random.choice([Cp_i[i], Cp_i[i] + np.random.randint(LOWER_BOUND_production, UPPER_BOUND_production)])

    Cs_k = np.random.randint(LOWER_BOUND_service, UPPER_BOUND_service, size = n) #затраты на единицу продукции для k узла спроса
    Ct_ij = np.random.randint(LOWER_BOUND_transport, UPPER_BOUND_transport, size = (m, 2)) #транспортные затраты на единицу продукции от i производственного центра к j распределительному центру
    Cr_jk = np.random.randint(LOWER_BOUND_transport, UPPER_BOUND_transport, size = (2, n)) #транспортные затраты на единицу продукции от j распределительного центра к k узлу спроса
    Cr_jk_transposed = np.transpose(Cr_jk)
    C_stop_i = np.random.randint(LOWER_BOUND_production, UPPER_BOUND_production, size = m) # затраты на остановку производства i за единицу продукции
    C_over_do_k = np.random.randint(LOWER_BOUND_service, UPPER_BOUND_service) # затраты на единицу лишней продукции
    extra_charge_percent = np.random.randint(LOWER_BOUND_extra_charge, UPPER_BOUND_extra_charge, size = n)
    extra_charge = extra_charge_percent/100 # наценка узла спроса

    alpha_prop = np.random.randint(LOWER_BOUND_safety_stock, UPPER_BOUND_safety_stock, size = m)/100  # вместимость страхового запаса в производственном узле i
    beta_prop = np.random.randint(LOWER_BOUND_safety_stock, UPPER_BOUND_safety_stock, size = 2)/100 # вместимость страхового запаса в распределительном узле j
    gamma_prop = np.random.randint(LOWER_BOUND_safety_stock, UPPER_BOUND_safety_stock, size = n)/100 # вместимость страхового запаса в узле спроса k
    
    alpha = np.floor(alpha_prop * nu)
    beta = np.floor(beta_prop * mu)
    gamma = np.floor(gamma_prop * d_k)

    Hp_i = np.random.randint(LOWER_BOUND_service_add, UPPER_BOUND_service_add, size = m) # стоимость хранения единицы продукции в производственном узле i в единицу времени
    Hd_j = np.random.randint(LOWER_BOUND_service_add, UPPER_BOUND_service_add, size = 2)# стоимость хранения единицы продукции в распределительном узле j в единицу времени
    Hs_k = np.random.randint(LOWER_BOUND_service_add, UPPER_BOUND_service_add, size = n)# стоимость хранения единицы продукции в узле спроса k в единицу времени
    
    # alpha = np.zeros(m, dtype = int)
    # beta = np.zeros(2, dtype = int)
    # gamma = np.zeros(n, dtype = int)


    data = {
        "d_k": d_k,
        "D": D,
        "mu": mu,
        "mu_add": mu_add,
        "nu": nu,
        "nu_add": nu_add,
        "Cp_i": Cp_i,
        "Cd_j": Cd_j,
        "Cd_j_add": Cd_j_add,
        "Cp_i_add": Cp_i_add,
        "Cs_k": Cs_k,
        "Ct_ij": Ct_ij,
        "Cr_jk": Cr_jk,
        "Cr_jk_transposed": Cr_jk_transposed,
        'C_stop_i': C_stop_i,
        'extra_charge': extra_charge,
        'alpha': alpha,
        'beta': beta,
        'gamma': gamma,
        'C_over_do_k': C_over_do_k,
        'Hp_i': Hp_i,
        'Hd_j': Hd_j,
        'Hs_k': Hs_k
    }

    #if verbose:
        #print("Сгенерированные данные:")
        #for key, value in data.items():
            #print(f"{key}: {value}")
    
    return data

In [40]:
def convert_numpy_to_python(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, (np.int32, np.int64)):
        return int(obj)
    elif isinstance(obj, dict):
        return {key: convert_numpy_to_python(value) for key, value in obj.items()}
    else:
        return obj
    
def flatten_data(data):
    flat_data = {}
    for key, value in data.items():
        if isinstance(value, list):
            # Если элемент - матрица
            if all(isinstance(x, list) for x in value):
                for i, row in enumerate(value):
                    for j, val in enumerate(row):
                        flat_data[f"{key}_{i}_{j}"] = val
            # Если просто список
            else:
                for i, val in enumerate(value):
                    flat_data[f"{key}_{i}"] = val
        # Если одиночное значение
        else:
            flat_data[key] = value
    return flat_data

In [41]:
#целочисленное линейное программирование 
def optimal_transportation(node_capacity, distr_capacity, cost_of_roads):
    try:
        # задача сбалансирована по условию (спрос = предлодению), проверка не нужна
        # if sum(node_capacity) != sum(distr_capacity):
        #     raise ValueError

        num_nodes = len(node_capacity)
        num_distr = len(distr_capacity)

        # задача минимизации
        prob = pulp.LpProblem("Transportation_Problem", pulp.LpMinimize)

        # матрица неотрицательный целочисленных переменных
        x = [[pulp.LpVariable(f"x_{i}_{j}", lowBound=0, cat='Integer') 
              for j in range(num_distr)] 
              for i in range(num_nodes)]

        # целевая функция: сумма стоимостей * x_ij (ее хотим минимизировать)
        prob += pulp.lpSum(cost_of_roads[i][j] * x[i][j] 
                          for i in range(num_nodes) 
                          for j in range(num_distr))

        # ограничения:
        # 1. сумма перевозок от узла i = его мощности
        for i in range(num_nodes):
            prob += pulp.lpSum(x[i][j] for j in range(num_distr)) == node_capacity[i]

        # 2. сумма перевозок в РЦ j = его вместимости
        for j in range(num_distr):
            prob += pulp.lpSum(x[i][j] for i in range(num_nodes)) == distr_capacity[j]

        prob.solve()

        if pulp.LpStatus[prob.status] != 'Optimal':
            print("Оптимальное целочисленное решение не найдено.")
            return None, None

        # матрица перевозок и общая стоимость
        transport_matrix = [[int(pulp.value(x[i][j])) for j in range(num_distr)] 
                           for i in range(num_nodes)]
        total_cost = int(pulp.value(prob.objective))

        return total_cost, transport_matrix

    except Exception as e:
        print(f"Ошибка: {e}")
        return None, None

In [42]:
# #черновик без функций 
# def distribution_node_disruption (m, n, index_of_broken_distr_node, data):
#     index_of_working_distr_node = 1 - index_of_broken_distr_node
#     transp_matrix_without_broken_1 = [row[index_of_working_distr_node] for row in data['q_ij']]
#     transp_matrix_with_broken_1 = [row[index_of_broken_distr_node] for row in data['q_ij']]
#     transp_matrix_without_broken_2 = [row[index_of_working_distr_node] for row in data['d_star_jk']]
#     Ct_ij = [row[index_of_working_distr_node] for row in data['Ct_ij']]
#     Cr_jk = [row[index_of_working_distr_node] for row in data['Cr_jk_transposed']]
#     print(sum(np.array(data['C_stop_i']) * transp_matrix_with_broken_1))

#     # 1. использование количества продукции, которое обычно поступает в рабочий рц
#     print("1. использование количества продукции, которое обычно поступает в рабочий рц")
#     # сценарий 1: получают те, которые привязаны к рабочему РЦ
#     total_cost_skript_1 = np.round(sum(transp_matrix_without_broken_1 * np.array(data['Cp_i'])) + (np.array(data['Cd_j'][index_of_working_distr_node]) * sum(transp_matrix_without_broken_1)) + 
#                   sum(np.array(data['Cs_k']) * transp_matrix_without_broken_2) + sum( x * y for x, y in zip(Ct_ij, transp_matrix_without_broken_1)) + 
#                   sum( x * y for x, y in zip(Cr_jk, transp_matrix_without_broken_2)) + sum(np.array(data['C_stop_i']) * transp_matrix_with_broken_1), decimals = 2)
#     total_value_skript_1 = np.round(sum(np.array(data['s_k']) * transp_matrix_without_broken_2), decimals = 2)
#     total_profit_skript_1 = np.round(total_value_skript_1 - total_cost_skript_1, decimals = 2)
#     print('сценарий 1: получают те, которые привязаны к рабочему РЦ:', total_profit_skript_1)    

   

#     sorted_indices = sorted(range(n), key = lambda i:(-data['s_k'][i]))
#     # сценарий 2: каждому узлу спроса одинаковое количество продукции
#     a = int(sum(transp_matrix_without_broken_1) / n)
#     if all(x > a for x in data['d_k']):
#         remainder_after_dividing_equally = sum(transp_matrix_without_broken_1) - a * n # его надо куда-то отправть, пусть самому выгодному 
#         add_q = np.zeros(n)
#         for k in sorted_indices:
#             if data['d_k'][k] - a >= 0 and remainder_after_dividing_equally != 0:
#                 if data['d_k'][k] - a > remainder_after_dividing_equally:
#                     add_q[k] = remainder_after_dividing_equally
#                     remainder_after_dividing_equally = 0
#                 else:
#                     add_q[k] = data['d_k'][k] - a
#                     remainder_after_dividing_equally -= (data['d_k'][k] - a)
#         a = add_q + a
#         total_cost_skript_2 = np.round(sum(transp_matrix_without_broken_1 * np.array(data['Cp_i'])) + (np.array(data['Cd_j'][index_of_working_distr_node]) * sum(transp_matrix_without_broken_1)) + 
#                   sum(np.array(data['Cs_k']) * a) + sum( x * y for x, y in zip(Ct_ij, transp_matrix_without_broken_1)) + 
#                   sum(x * y for x, y in zip(a, Cr_jk)) + sum(np.array(data['C_stop_i']) * transp_matrix_with_broken_1), decimals = 2)
#         total_value_skript_2 = np.round(sum(a * (np.array(data['s_k']))), decimals = 2)
#         total_profit_skript_2 = np.round(total_value_skript_2 - total_cost_skript_2, decimals = 2)
#         print('сценарий 2: каждому узлу спроса одинаковое количество продукции:', total_profit_skript_2)  
#     else:
#         print('не можем использовать сценарий 2: каждому узлу спроса одинаковое количество продукции')

#     #сценарий 3: пропорционально потребностям узлов спроса, используя весовые коэффициенты
#     W = np.array(data['d_k']) / data['D']
#     prop = W * sum(transp_matrix_without_broken_1)
#     int_prop = [math.floor(p) for p in prop]
#     remainder_after_weighting_factors = sum(transp_matrix_without_broken_1) - sum(int_prop)
#     add_q_2 = np.zeros(n)
#     for k in sorted_indices:
#         if data['d_k'][k] - int_prop[k] > 0 and remainder_after_weighting_factors!= 0:
#             if data['d_k'][k] - int_prop[k] > remainder_after_weighting_factors:
#                 add_q_2[k] = remainder_after_weighting_factors
#                 remainder_after_weighting_factors = 0
#             else:
#                 add_q_2[k] = data['d_k'][k] - int_prop[k]
#                 remainder_after_weighting_factors -= (data['d_k'][k] - int_prop[k])

#     int_prop = add_q_2 + int_prop # столько идет узлам 
#     total_cost_skript_3 = np.round(sum(transp_matrix_without_broken_1 * np.array(data['Cp_i'])) + (np.array(data['Cd_j'][index_of_working_distr_node]) * sum(transp_matrix_without_broken_1)) + 
#                   sum(np.array(data['Cs_k']) * int_prop) + sum( x * y for x, y in zip(Ct_ij, transp_matrix_without_broken_1)) + 
#                   sum(x * y for x, y in zip(int_prop, Cr_jk)) + sum(np.array(data['C_stop_i']) * transp_matrix_with_broken_1), decimals = 2)
    
#     total_value_skript_3 = np.round(sum(int_prop * (np.array(data['s_k']))), decimals = 2)
#     total_profit_skript_3 = np.round(total_value_skript_3 - total_cost_skript_3, decimals = 2)
#     print('сценарий 3: пропорционально потребностям узлов спроса, используя весовые коэффициенты:', total_profit_skript_3)   

#     # сценарий 4: cортировка узлов спроса по наибольшей выручке для максимизации прибыли
#     h = np.zeros(n)
#     filling_demand_nodes = np.zeros(n)
#     remainder =  sum(transp_matrix_without_broken_1)
#     for k in range(n):
#         h[k] = data['s_k'][k] - (data['Cs_k'][k] + Cr_jk[k]) # величина, по которой можно определить, какой магазин, продавая товары, будет приносить наибольшую прибыль
#     sorted_indices_beneficial = sorted(range(n), key = lambda k: -h[k])
#     for k in sorted_indices_beneficial:
#         if data['d_k'][k] > remainder and remainder != 0:
#             filling_demand_nodes[k] = remainder
#             remainder = 0
#         elif data['d_k'][k] < remainder and remainder != 0:
#             filling_demand_nodes[k] = data['d_k'][k]
#             remainder -= data['d_k'][k]

#     total_cost_skript_4 = np.round(sum(transp_matrix_without_broken_1 * np.array(data['Cp_i'])) + (np.array(data['Cd_j'][index_of_working_distr_node]) * sum(transp_matrix_without_broken_1)) + 
#                   sum(np.array(data['Cs_k']) * filling_demand_nodes) + sum( x * y for x, y in zip(Ct_ij, transp_matrix_without_broken_1)) + 
#                   sum(x * y for x, y in zip(filling_demand_nodes, Cr_jk)) + sum(np.array(data['C_stop_i']) * transp_matrix_with_broken_1), decimals = 2)
    
#     total_value_skript_4 = np.round(sum(filling_demand_nodes * (np.array(data['s_k']))), decimals = 2)
#     total_profit_skript_4 = np.round(total_value_skript_4 - total_cost_skript_4, decimals = 2)
#     print('сценарий 4: cортировка узлов спроса по наибольшей выручке для максимизации прибыли:', total_profit_skript_4)   

#     #сценарий 5: сортировка про выручке за единицу в узле
#     h_profit = np.zeros(n)
    
#     filling_demand_nodes_profit = np.zeros(n)
#     remainder_profit =  sum(transp_matrix_without_broken_1)
#     sorted_indices_beneficial_profit = sorted(range(n), key = lambda i: -data['profit_per_unit'][i])
#     for k in sorted_indices_beneficial_profit:
#         if data['d_k'][k] > remainder_profit and remainder_profit != 0:
#             filling_demand_nodes_profit[k] = remainder_profit
#             remainder_profit = 0
#         elif data['d_k'][k] < remainder_profit and remainder_profit != 0:
#             filling_demand_nodes_profit[k] = data['d_k'][k]
#             remainder_profit -= data['d_k'][k]
#     total_cost_skript_5 = np.round(sum(transp_matrix_without_broken_1 * np.array(data['Cp_i'])) + (np.array(data['Cd_j'][index_of_working_distr_node]) * sum(transp_matrix_without_broken_1)) + 
#                   sum(np.array(data['Cs_k']) * filling_demand_nodes_profit) + sum( x * y for x, y in zip(Ct_ij, transp_matrix_without_broken_1)) + 
#                   sum(x * y for x, y in zip(filling_demand_nodes_profit, Cr_jk)) + sum(np.array(data['C_stop_i']) * transp_matrix_with_broken_1), decimals = 2)
    
#     total_value_skript_5 = np.round(sum(filling_demand_nodes_profit * (np.array(data['s_k']))), decimals = 2)
#     total_profit_skript_5 = np.round(total_value_skript_5 - total_cost_skript_5, decimals = 2)
#     print('сценарий 5: сортировка про выручке за единицу в узле:', total_profit_skript_5)   

#     # 2. использование перенаправления 
#     # сценарий 6: перенаправление всей продукции в рабочий рц
#     if ((np.array(data['mu']) + np.array(data['mu_add'])))[index_of_working_distr_node] >= data['D']:
#         total_cost_skript_6 = np.round(sum(np.array(data['nu']) * np.array(data['Cp_i'])) + (np.array(data['Cd_j'][index_of_working_distr_node] + np.array(data['Cd_j_add'][index_of_working_distr_node]) * data['D'])) + 
#                   sum(np.array(data['Cs_k']) * np.array(data['d_k']) ) + sum( x * y for x, y in zip(Ct_ij, data['nu'])) + 
#                   sum( x * y for x, y in zip(Cr_jk, data['d_k'])), decimals = 2)
#         total_value_skript_6 = np.round(sum(np.array(data['s_k']) * np.array(data['d_k'])), decimals = 2)
#         total_profit_skript_6 = np.round(total_value_skript_6 - total_cost_skript_6, decimals = 2)
#         print('сценарий 6: перенаправляем все в рабочий рц:', total_profit_skript_6)  
#     else:
#         print("не можем использовать сценарий 6: перенаправление всей продукции в рабочий рц")

#     # сценарий 7: перенаправление продукции с некоторых производственных узлов в рабочий рц
#     if ((np.array(data['mu']) + np.array(data['mu_add'])))[index_of_working_distr_node] < data['D']:
#         u = np.zeros(m)
#         add_prod = np.zeros(m)
#         for i in range(m):
#             u[i] = np.array(data['Cp_i'])[i] +  (np.array(Ct_ij))[i] - np.array(data['C_stop_i'])[i]
#         sorted_prod_indices = sorted(range(m), key = lambda i: -u[i])
#         print(u, sorted_prod_indices)
#         remainder_add_mu = data['mu_add'][index_of_working_distr_node]
#         for i in sorted_prod_indices:
#             if (data['nu'][i] - transp_matrix_without_broken_1[i]) < remainder_add_mu and remainder_add_mu !=0:
#                 add_prod[k] = data['nu'][i]
#                 remainder_add_mu -= data['nu'][i]
#             elif (data['nu'][i] - -transp_matrix_without_broken_1[i]) >= remainder_add_mu and remainder_add_mu !=0:
#                 add_prod[i] = remainder_add_mu
#                 remainder_add_mu = 0
#         print(add_prod)
#         transp_matrix_add_prod = transp_matrix_with_broken_1 + add_prod
        
        

















In [ ]:
n = 4  # число узлов спроса
m = 3  # число производственных центров
generated_data =  generate_data(n, m)
#распределяем продукцию из пц по рц
transp_cost_between_prod_and_distr, transp_matrix_between_prod_and_distr = optimal_transportation( generated_data["nu"], generated_data["mu"], generated_data["Ct_ij"])
transp_cost_between_distr_and_demand, transp_matrix_between_distr_and_demand = optimal_transportation( generated_data["d_k"], generated_data["mu"], generated_data["Cr_jk_transposed"])
data = convert_numpy_to_python(generated_data)

# prod_cost_of_unit = np.round((sum(nu * Cp_i for nu, Cp_i in zip(data['nu'], data['Cp_i']))/data['D']), decimals = 2) # взвешенная стоимость производства единицы продукции
# prod_transp_between_prod_and_distr_of_unit = np.round((
#     np.sum(np.array(transp_matrix_between_prod_and_distr) * np.array(data['Ct_ij'])) / data['D']), decimals = 2) # взвешенная стоимость доставки из ПЦ в РЦ

# cost_in_d_for_k = np.zeros(n, dtype = int) # затраты в РЦ на единицу товара
# for k in range(n):
#     cost_row = np.array(data['Cd_j'])
#     if transp_matrix_between_distr_and_demand[k][0] > 0 and transp_matrix_between_distr_and_demand[k][1] > 0:
#         cost_in_d_for_k[k] =   np.round((np.sum(np.array(transp_matrix_between_distr_and_demand[k]) * np.array(data['Cd_j']))/ data['d_k'][k]), decimals = 2)
#     if transp_matrix_between_distr_and_demand[k][0] > 0 and transp_matrix_between_distr_and_demand[k][1] ==0 :
#         cost_in_d_for_k[k] = cost_row[0]
#     if transp_matrix_between_distr_and_demand[k][0] == 0 and transp_matrix_between_distr_and_demand[k][1] > 0 :
#         cost_in_d_for_k[k] = cost_row[1]
# HE = cost_in_d_for_k + prod_cost_of_unit + prod_transp_between_prod_and_distr_of_unit
# cost_price = np.array (cost_in_d_for_k + data['Cs_k'] + prod_cost_of_unit + prod_transp_between_prod_and_distr_of_unit) #себестоимость единицы продукции в k узле спроса
# s_k = np.round(cost_price * (1 + np.array(data['extra_charge'])), decimals = 2)


E_d = np.zeros(2) # стоимость единицы товара в j распределительном центре
E_s = np.zeros(n) # стоимость единицы товара в k узле спроса
sum_1 = np.zeros(m)
sum_2 = np.zeros(2)
dot_2 = np.zeros(n)
for j in range(2):
    for i in range(m):
        sum_1[i] = (data['Cp_i'][i] + data['Ct_ij'][i][j] + data['Cd_j'][j])* transp_matrix_between_prod_and_distr[i][j]
    E_d[j] = np.round((np.sum(sum_1))/data['mu'][j], decimals = 2)
for k in range(n):
    for j in range(2):
        sum_2[j] = ((E_d[j] + data['Cr_jk'][j][k] + data['Cs_k'][k]))
    dot_2 = sum_2 * transp_matrix_between_distr_and_demand[k]
    E_s[k] = np.round(np.sum(dot_2)/data['d_k'][k], decimals = 2)
s_k = np.round(E_s * (1 + np.array(data['extra_charge'])), decimals = 2)


profit_per_unit = np.round(s_k - E_s, decimals = 2)
cost_1 = sum(np.array(data['nu']) * np.array(data['Cp_i']))
cost_2 = sum(np.array(data['Cd_j']) * np.array(data['mu']))
cost_3 = sum(np.array(data['Cs_k']) * np.array(data['d_k']))
cost_4 = sum(sum(np.array(data['Ct_ij']) * transp_matrix_between_prod_and_distr))
cost_5 = sum(sum(np.array(data['Cr_jk_transposed']) * transp_matrix_between_distr_and_demand))
total_cost = cost_1 + cost_2 + cost_3 + cost_4 + cost_5
print('total_cost', total_cost)
total_value = np.round(sum(s_k * np.array(data['d_k'])), decimals = 2)
print('total_value', total_value)
total_profit = np.round(total_value - total_cost, decimals = 2)
print('total_profit', total_profit) 


all_gen_data = {
    "d_k": data["d_k"],
    "D": data["D"],
    "mu": data["mu"],
    "mu_add": data["mu_add"],
    "nu": data["nu"],
    "nu_add": data["nu_add"],
    "Cp_i": data["Cp_i"],
    "Cd_j": data["Cd_j"],
    "Cp_i_add": data['Cp_i_add'],
    "Cd_j_add": data["Cd_j_add"],
    "Cs_k": data["Cs_k"],
    "Ct_ij": data["Ct_ij"],
    "Cr_jk": data["Cr_jk"],
    "Cr_jk_transposed": data["Cr_jk_transposed"],
    "extra_charge": data["extra_charge"],
    'C_stop_i': data["C_stop_i"],
    
    'q_ij': transp_matrix_between_prod_and_distr,
    "d_star_jk": transp_matrix_between_distr_and_demand,
    "C_over_do_k": data['C_over_do_k'],
    "s_k": s_k.tolist(),
    "profit_per_unit": profit_per_unit,
    "E_d": E_d,
    "E_s": E_s, 
    'Hp_i': data['Hp_i'],
    'Hd_j': data['Hd_j'],
    'Hs_k': data['Hs_k'],
    'alpha': data["alpha"],
    'beta': data["beta"],
    'gamma': data["gamma"]
}
for key, value in all_gen_data.items():
    print(f"{key}: {value}")

# flat_data = flatten_data(data_to_export)
# with open('supply_chain_data.csv', 'w', newline='') as csvfile:
#     writer = csv.writer(csvfile)
#     writer.writerow(flat_data.keys())
#     writer.writerow(flat_data.values())




total_cost 101320
total_value 131456.07
total_profit 30136.07
d_k: [317, 376, 380, 350]
D: 1423
mu: [598, 825]
mu_add: [598, 82]
nu: [703, 502, 218]
nu_add: [70, 50, 218]
Cp_i: [46, 34, 20]
Cd_j: [5, 12]
Cp_i_add: [82, 83, 20]
Cd_j_add: [0, 12]
Cs_k: [8, 9, 11, 12]
Ct_ij: [[3, 2], [11, 17], [9, 7]]
Cr_jk: [[12, 17, 18, 1], [8, 11, 10, 4]]
Cr_jk_transposed: [[12, 8], [17, 11], [18, 10], [1, 4]]
extra_charge: [0.22, 0.34, 0.26, 0.37]
C_stop_i: [37, 24, 48]
q_ij: [[96, 607], [502, 0], [0, 218]]
d_star_jk: [[248, 69], [0, 376], [0, 380], [350, 0]]
C_over_do_k: 14
s_k: [86.13, 99.76, 95.07, 87.19]
profit_per_unit: [15.53 25.31 19.62 23.55]
E_d: [50.64 54.45]
E_s: [70.6  74.45 75.45 63.64]
Hp_i: [10, 6, 13]
Hd_j: [12, 15]
Hs_k: [17, 8, 28, 20]
alpha: [456.0, 5.0, 167.0]
beta: [304.0, 280.0]
gamma: [60.0, 364.0, 159.0, 196.0]


In [44]:
# способы распределения продукции по узлам, когда предложение не удовлетворяет спросу, при сбое в РЦ 
# сценарий 1: получают те, которые привязаны к рабочему РЦ
def scenario_1(transp_matrix_without_broken_1, transp_matrix_without_broken_2, 
               transp_matrix_with_broken_1, data, Ct_ij, Cr_jk, index_of_working_distr_node, n):
    total_cost = np.round(
        sum(transp_matrix_without_broken_1 * np.array(data['Cp_i'])) + 
        (np.array(data['Cd_j'][index_of_working_distr_node]) * sum(transp_matrix_without_broken_1)) + 
        sum(np.array(data['Cs_k']) * transp_matrix_without_broken_2) + 
        sum(x * y for x, y in zip(Ct_ij, transp_matrix_without_broken_1)) + 
        sum(x * y for x, y in zip(Cr_jk, transp_matrix_without_broken_2)) + 
        sum(np.array(data['C_stop_i']) * transp_matrix_with_broken_1), decimals=2)
    
    total_value = np.round(sum(np.array(data['s_k']) * transp_matrix_without_broken_2), decimals=2)
    total_profit = np.round(total_value - total_cost, decimals=2)
    return total_profit, transp_matrix_without_broken_2

# сценарий 2: каждому узлу спроса одинаковое количество продукции
def scenario_2(transp_matrix_without_broken_1, transp_matrix_with_broken_1, 
              data, Ct_ij, Cr_jk, index_of_working_distr_node, n):
    h = np.zeros(n)
    for k in range(n):
        h[k] = data['s_k'][k] - (data['Cs_k'][k] + Cr_jk[k])
    sorted_indices = sorted(range(n), key=lambda i: (-h[i]))
    a = int(sum(transp_matrix_without_broken_1) / n)
    
    if all(x > a for x in data['d_k']):
        remainder = sum(transp_matrix_without_broken_1) - a * n
        add_q = np.zeros(n)
        
        for k in sorted_indices:
            if data['d_k'][k] - a >= 0 and remainder != 0:
                if data['d_k'][k] - a > remainder:
                    add_q[k] = remainder
                    remainder = 0
                else:
                    add_q[k] = data['d_k'][k] - a
                    remainder -= (data['d_k'][k] - a)
        
        a = add_q + a
        total_cost = np.round(
            sum(x * y for x, y in zip(transp_matrix_without_broken_1, np.array(data['Cp_i']))) + 
            (np.array(data['Cd_j'][index_of_working_distr_node]) * sum(transp_matrix_without_broken_1)) + 
            sum(np.array(data['Cs_k']) * a) + 
            sum(x * y for x, y in zip(Ct_ij, transp_matrix_without_broken_1)) + 
            sum(x * y for x, y in zip(a, Cr_jk)) + 
            sum(np.array(data['C_stop_i']) * transp_matrix_with_broken_1), decimals=2)
        
        total_value = np.round(sum(a * (np.array(data['s_k']))), decimals=2)
        total_profit = np.round(total_value - total_cost, decimals=2)
        return total_profit, a
    else:
        return None, None
    
#сценарий 3: пропорционально потребностям узлов спроса, используя весовые коэффициенты
def scenario_3(transp_matrix_without_broken_1, transp_matrix_with_broken_1, 
              data, Ct_ij, Cr_jk, index_of_working_distr_node, n):
    h = np.zeros(n)
    for k in range(n):
        h[k] = data['s_k'][k] - (data['Cs_k'][k] + Cr_jk[k])
    
    sorted_indices = sorted(range(n), key=lambda k: -h[k])
    W = np.array(data['d_k']) / data['D']
    prop = W * sum(transp_matrix_without_broken_1)
    int_prop = [math.floor(p) for p in prop]
    remainder = sum(transp_matrix_without_broken_1) - sum(int_prop)
    add_q = np.zeros(n)
    
    for k in sorted_indices:
        if data['d_k'][k] - int_prop[k] > 0 and remainder != 0:
            if data['d_k'][k] - int_prop[k] > remainder:
                add_q[k] = remainder
                remainder = 0
            else:
                add_q[k] = data['d_k'][k] - int_prop[k]
                remainder -= (data['d_k'][k] - int_prop[k])
    
    int_prop = add_q + int_prop
    total_cost = np.round(
        sum(x * y for x, y in zip(transp_matrix_without_broken_1, np.array(data['Cp_i']))) + 
        (np.array(data['Cd_j'][index_of_working_distr_node]) * sum(transp_matrix_without_broken_1)) + 
        sum(np.array(data['Cs_k']) * int_prop) + 
        sum(x * y for x, y in zip(Ct_ij, transp_matrix_without_broken_1)) + 
        sum(x * y for x, y in zip(int_prop, Cr_jk)) + 
        sum(np.array(data['C_stop_i']) * transp_matrix_with_broken_1), decimals=2)
    
    total_value = np.round(sum(int_prop * (np.array(data['s_k']))), decimals=2)
    total_profit = np.round(total_value - total_cost, decimals=2)
    return total_profit, int_prop

# сценарий 4: cортировка узлов спроса по наибольшей выручке для максимизации прибыли
def scenario_4(transp_matrix_without_broken_1, transp_matrix_with_broken_1, 
              data, Ct_ij, Cr_jk, index_of_working_distr_node, n):
    h = np.zeros(n)
    filling = np.zeros(n)
    remainder = sum(transp_matrix_without_broken_1)
    
    for k in range(n):
        h[k] = data['s_k'][k] - (data['Cs_k'][k] + Cr_jk[k])
    
    sorted_indices = sorted(range(n), key=lambda k: -h[k])
    
    for k in sorted_indices:
        if data['d_k'][k] > remainder and remainder != 0:
            filling[k] = remainder
            remainder = 0
        elif data['d_k'][k] < remainder and remainder != 0:
            filling[k] = data['d_k'][k]
            remainder -= data['d_k'][k]
    
    total_cost = np.round(
        sum(transp_matrix_without_broken_1 * np.array(data['Cp_i'])) + 
        (np.array(data['Cd_j'][index_of_working_distr_node]) * sum(transp_matrix_without_broken_1)) + 
        sum(np.array(data['Cs_k']) * filling) + 
        sum(x * y for x, y in zip(Ct_ij, transp_matrix_without_broken_1)) + 
        sum(x * y for x, y in zip(filling, Cr_jk)) + 
        sum(np.array(data['C_stop_i']) * transp_matrix_with_broken_1), decimals=2)
    
    total_value = np.round(sum(filling * (np.array(data['s_k']))), decimals=2)
    total_profit = np.round(total_value - total_cost, decimals=2)
    return total_profit, filling

#сценарий 5: сортировка про выручке за единицу в узле
def scenario_5(transp_matrix_without_broken_1, transp_matrix_with_broken_1, 
              data, Ct_ij, Cr_jk, index_of_working_distr_node, n):
    filling = np.zeros(n)
    remainder = sum(transp_matrix_without_broken_1)
    sorted_indices = sorted(range(n), key=lambda i: -data['profit_per_unit'][i])
    
    for k in sorted_indices:
        if data['d_k'][k] > remainder and remainder != 0:
            filling[k] = remainder
            remainder = 0
        elif data['d_k'][k] < remainder and remainder != 0:
            filling[k] = data['d_k'][k]
            remainder -= data['d_k'][k]
    
    total_cost = np.round(
        sum(transp_matrix_without_broken_1 * np.array(data['Cp_i'])) + 
        (np.array(data['Cd_j'][index_of_working_distr_node]) * sum(transp_matrix_without_broken_1)) + 
        sum(np.array(data['Cs_k']) * filling) + 
        sum(x * y for x, y in zip(Ct_ij, transp_matrix_without_broken_1)) + 
        sum(x * y for x, y in zip(filling, Cr_jk)) + 
        sum(np.array(data['C_stop_i']) * transp_matrix_with_broken_1), decimals=2)
    
    total_value = np.round(sum(filling * (np.array(data['s_k']))), decimals=2)
    total_profit = np.round(total_value - total_cost, decimals=2)
    return total_profit, filling




In [45]:
def distribution_node_disruption(m, n, index_of_broken_distr_node, data):
    index_of_working_distr_node = 1 - index_of_broken_distr_node
    transp_matrix_without_broken_1 = [row[index_of_working_distr_node] for row in data['q_ij']]
    transp_matrix_with_broken_1 = [row[index_of_broken_distr_node] for row in data['q_ij']]
    transp_matrix_without_broken_2 = [row[index_of_working_distr_node] for row in data['d_star_jk']]
    Ct_ij = [row[index_of_working_distr_node] for row in data['Ct_ij']]
    Cr_jk = [row[index_of_working_distr_node] for row in data['Cr_jk_transposed']]
    #1. использование количества продукции, которое обычно поступает в рабочий рц
    print("1. использование количества продукции, которое обычно поступает в рабочий рц")
    profit1, dist1 = scenario_1(transp_matrix_without_broken_1, transp_matrix_without_broken_2,
                              transp_matrix_with_broken_1, data, Ct_ij, Cr_jk, index_of_working_distr_node, n)
    print('сценарий 1:', profit1)
    
    profit2, dist2 = scenario_2(transp_matrix_without_broken_1, transp_matrix_with_broken_1,
                              data, Ct_ij, Cr_jk, index_of_working_distr_node, n)
    if profit2 is not None:
        print('сценарий 2:', profit2)
    else:
        print('не можем использовать сценарий 2:')
    
    profit3, dist3 = scenario_3(transp_matrix_without_broken_1, transp_matrix_with_broken_1,
                              data, Ct_ij, Cr_jk, index_of_working_distr_node, n)
    print('сценарий 3:', profit3)
    
    profit4, dist4 = scenario_4(transp_matrix_without_broken_1, transp_matrix_with_broken_1,
                              data, Ct_ij, Cr_jk, index_of_working_distr_node, n)
    print('сценарий 4:', profit4)
    
    profit5, dist5 = scenario_5(transp_matrix_without_broken_1, transp_matrix_with_broken_1,
                              data, Ct_ij, Cr_jk, index_of_working_distr_node, n)
    print('сценарий 5:', profit5)

    # 2. использование перенаправления 
    print("\n2. использование перенаправления ")
    # сценарий 6: перенаправление всей продукции в рабочий рц
    if ((np.array(data['mu']) + np.array(data['mu_add'])))[index_of_working_distr_node] >= data['D']:
        total_cost_skript_6 = np.round(sum(np.array(data['nu']) * np.array(data['Cp_i'])) + (np.array(data['Cd_j'][index_of_working_distr_node] + np.array(data['Cd_j_add'][index_of_working_distr_node]) * data['D'])) + 
                  sum(np.array(data['Cs_k']) * np.array(data['d_k']) ) + sum( x * y for x, y in zip(Ct_ij, data['nu'])) + 
                  sum( x * y for x, y in zip(Cr_jk, data['d_k'])), decimals = 2)
        total_value_skript_6 = np.round(sum(np.array(data['s_k']) * np.array(data['d_k'])), decimals = 2)
        total_profit_skript_6 = np.round(total_value_skript_6 - total_cost_skript_6, decimals = 2)
        print('сценарий 6: перенаправляем все в рабочий рц:', total_profit_skript_6)  
    else:
        print("не можем использовать сценарий 6: перенаправление всей продукции в рабочий рц")


    # сценарий 7: перенаправление продукции с некоторых производственных узлов 
    if ((np.array(data['mu']) + np.array(data['mu_add'])))[index_of_working_distr_node] < data['D']:
        u = np.zeros(m)
        add_prod = np.zeros(m)
        for i in range(m):
            u[i] = np.array(data['Cp_i'])[i] + (np.array(Ct_ij))[i] - np.array(data['C_stop_i'])[i]
        
        sorted_prod_indices = sorted(range(m), key=lambda i: -u[i])
        remainder_add_mu = data['mu_add'][index_of_working_distr_node]
        
        for i in sorted_prod_indices:
            if (data['nu'][i] - transp_matrix_without_broken_1[i]) < remainder_add_mu and remainder_add_mu != 0:
                add_prod[i] = data['nu'][i]
                remainder_add_mu -= data['nu'][i]
            elif (data['nu'][i] - transp_matrix_without_broken_1[i]) >= remainder_add_mu and remainder_add_mu != 0:
                add_prod[i] = remainder_add_mu
                remainder_add_mu = 0
        new_transp_matrix = transp_matrix_without_broken_1 + add_prod

        print("сценарий 7: перенаправление продукции с некоторых производственных узлов: ")
        profit1_7, _ = scenario_1(new_transp_matrix, transp_matrix_without_broken_2, transp_matrix_with_broken_1 - add_prod, data, Ct_ij, Cr_jk, index_of_working_distr_node, n)
        print('сценарий 7.1:', profit1_7)
        profit2_7, _ = scenario_2(new_transp_matrix, transp_matrix_with_broken_1 - add_prod, data, Ct_ij, Cr_jk, index_of_working_distr_node, n)
        if profit2_7 is not None:
            print('сценарий 7.2:', profit2_7)
        else: 
            print('не можем использовать сценарий 7.2')
        profit3_7, _ = scenario_3(new_transp_matrix, transp_matrix_with_broken_1 - add_prod, data, Ct_ij, Cr_jk, index_of_working_distr_node, n)
        print('сценарий 7.3:', profit3_7)
        profit4_7, _ = scenario_4(new_transp_matrix, transp_matrix_with_broken_1 - add_prod, data, Ct_ij, Cr_jk, index_of_working_distr_node, n)
        print('сценарий 7.4:', profit4_7)
        profit5_7, _ = scenario_5(new_transp_matrix, transp_matrix_with_broken_1 - add_prod, data, Ct_ij, Cr_jk, index_of_working_distr_node, n)
        print('сценарий 7.5:', profit5_7)
        

In [54]:
RC = 0
d_2 = distribution_node_disruption(m, n, RC, all_gen_data)

1. использование количества продукции, которое обычно поступает в рабочий рц
сценарий 1: 2453.33
сценарий 2: 358.66
сценарий 3: 534.77
сценарий 4: 2526.47
сценарий 5: 1717.19

2. использование перенаправления 
не можем использовать сценарий 6: перенаправление всей продукции в рабочий рц
сценарий 7: перенаправление продукции с некоторых производственных узлов: 
сценарий 7.1: -744.67
сценарий 7.2: 3223.18
сценарий 7.3: 3388.97
сценарий 7.4: 5166.05
сценарий 7.5: 4592.93


In [47]:
# # неправильный подход к узлам спроса при сбое в ПЦ
# def overflow_check(data, dist_edit, dist, profit, scenario_number):
#     sorted_indices = sorted(range(n), key=lambda k: -data['profit_per_unit'][k])
#     if any(data['d_k'] < dist_edit):
#         print(f'не можем использовать сценарий {scenario_number}')
#     else: 
#         print(f"сценарий {scenario_number}", profit)

#     # for k in sorted_indices:
#     #     if data['d_k'][k] < dist_edit[k]:
#     #         if k == n:
#     #             dist_edit[0] += dist_edit[k] - data['d_k'][k]
#     #         else: 
#     #             dist_edit[k+1] += dist_edit[k] - data['d_k'][k]
#     #         dist_edit[k] = data['d_k'][k]
#     # if not np.array_equal(dist, dist_edit):
#     #     print(f'не можем использовать сценарий{scenario_number}')
#     # else:
#     #     print(f"сценарий {scenario_number}", profit)
        

    
# def production_node_disruption(m, n, index_of_broken_prod_node, data):
#     # 1. используем количество продукции, которое производит (m - 1) производственный узел

#     print("1. используем количество продукции, которое производит (m - 1) производственный узел")
#     q_without_broken_prod_node = copy.deepcopy(data['q_ij'])
#     q_without_broken_prod_node[index_of_broken_prod_node] = [0, 0]
#     transp_matrix_stop = np.zeros(m) # так как не замедляем процессы 

#     # если ломается производство, которое поствляет в оба РЦ
#     if all(data['q_ij'][index_of_broken_prod_node]) !=0:
#         transp_matrix_1_for_first_PC = [row[0] for row in q_without_broken_prod_node]
#         if sum(transp_matrix_1_for_first_PC) == 0:
#             transp_matrix_2_for_first_PC = np.zeros(n)
#         else:
#             transp_matrix_2_for_first_PC = [row[0] for row in data['d_star_jk']]
#         Ct_ij_1 = [row[0] for row in data['Ct_ij']]
#         Cr_jk_1 = [row[0] for row in data['Cr_jk_transposed']]
#         index_of_working_distr_node = 0 

#         transp_matrix_1_for_second_PC = [row[1] for row in q_without_broken_prod_node]
#         if sum(transp_matrix_1_for_second_PC) == 0:
#             transp_matrix_2_for_second_PC = np.zeros(n)
#         else:
#             transp_matrix_2_for_second_PC = [row[1] for row in data['d_star_jk']]
#         Ct_ij_2 = [row[1] for row in data['Ct_ij']]
#         Cr_jk_2 = [row[1] for row in data['Cr_jk_transposed']]
#         index_of_working_distr_node_2 = 1


#         # profit_2_1, dist1_scenario_2 = scenario_2(transp_matrix_1_for_first_PC,
#         #                       transp_matrix_stop, data, Ct_ij_1, Cr_jk_1, index_of_working_distr_node, n)
#         # profit_2_2, dist2_scenario_2 = scenario_2(transp_matrix_1_for_second_PC,
#         #                       transp_matrix_stop, data, Ct_ij_2, Cr_jk_2, index_of_working_distr_node, n)
#         # dist_scenario_2 = dist1_scenario_2 + dist2_scenario_2
#         # dist_edit_scenario_2 = dist1_scenario_2 + dist2_scenario_2
#         # profit_2 = profit_2_1 + profit_2_2
#         # overflow_check_scenario_2 = overflow_check(data, dist_edit_scenario_2, dist_scenario_2, profit_2, 1.2)

#         # profit_3_1, dist1_scenario_3 = scenario_3(transp_matrix_1_for_first_PC,
#         #                       transp_matrix_stop, data, Ct_ij_1, Cr_jk_1, index_of_working_distr_node, n)
#         # profit_3_2, dist2_scenario_3 = scenario_3(transp_matrix_1_for_second_PC,
#         #                       transp_matrix_stop, data, Ct_ij_2, Cr_jk_2, index_of_working_distr_node, n)
#         # dist_scenario_3 = dist1_scenario_3 + dist2_scenario_3
#         # dist_edit_scenario_3 = dist1_scenario_3 + dist2_scenario_3
#         # profit_3 = profit_3_1 + profit_3_2
#         # overflow_check_scenario_3 = overflow_check(data, dist_edit_scenario_3, dist_scenario_3, profit_3, 1.3)

#         results = []
#         scenarios = [(scenario_2, 1.2), (scenario_3, 1.3), (scenario_4, 1.4), (scenario_5, 1.5)]
        
#         for scenario_func, scenario_num in scenarios:
#             profit_1, dist1 = scenario_func(transp_matrix_1_for_first_PC, 
#                                   transp_matrix_stop, data, 
#                                   Ct_ij_1, Cr_jk_1, 
#                                   index_of_working_distr_node, n)
#             profit_2, dist2 = scenario_func(transp_matrix_1_for_second_PC, 
#                                   transp_matrix_stop, data, 
#                                   Ct_ij_2, Cr_jk_2, 
#                                   index_of_working_distr_node, n)

#             total_dist = dist1 + dist2
#             total_profit = profit_1 + profit_2
    
#             overflow = overflow_check(data, total_dist, total_dist, total_profit, scenario_num)
    
#             results.append({
#                 'scenario': scenario_num,
#                 'profit': total_profit,
#                 'distribution': total_dist,
#                 'overflow_check': overflow })
#     # ломается производство, которое привязано только к одному рц
#     else:
#         print("ломается производство, которое привязано только к одному рц")
#         if data['q_ij'][index_of_broken_prod_node][0] != 0:
#             index_of_linked_distr_node = 0
#         else:
#             index_of_linked_distr_node = 1
#         transp_matrix_1 = [row[index_of_linked_distr_node] for row in q_without_broken_prod_node]
#         Ct_ij = [row[index_of_linked_distr_node] for row in data['Ct_ij']]
#         Cr_jk = [row[index_of_linked_distr_node] for row in data['Cr_jk_transposed']]
#         index_of_working_distr_node = index_of_linked_distr_node
        
        
            


        




In [48]:
# def production_node_disruption(m, n, index_of_broken_prod_node, data):
#     print(data['q_ij'])
#     # 1. используем количество продукции, которое производит (m - 1) производственный узел
#     print("1. используем количество продукции, которое производит (m - 1) производственный узел")
#     q_without_broken_prod_node = copy.deepcopy(data['q_ij'])
#     q_without_broken_prod_node[index_of_broken_prod_node] = [0, 0]
#     transp_matrix_stop = np.zeros(m) # так как не замедляем процессы 
#     w_1 = [row for i, row in enumerate(q_without_broken_prod_node) if i != index_of_broken_prod_node]
#     print(np.sum(w_1, axis=1))
#     # если ломается производство, которое поствляет в оба РЦ
#     if all(data['q_ij'][index_of_broken_prod_node]) !=0:
#         transp_matrix_1_for_first_PC = [row[0] for row in q_without_broken_prod_node]
#         if sum(transp_matrix_1_for_first_PC) == 0:
#             transp_matrix_2_for_first_PC = np.zeros(n)
#         else:
#             transp_matrix_2_for_first_PC = [row[0] for row in data['d_star_jk']]
#         Ct_ij_1 = [row[0] for row in data['Ct_ij']]
#         Cr_jk_1 = [row[0] for row in data['Cr_jk_transposed']]
#         index_of_working_distr_node_1 = 0 

#         transp_matrix_1_for_second_PC = [row[1] for row in q_without_broken_prod_node]
#         if sum(transp_matrix_1_for_second_PC) == 0:
#             transp_matrix_2_for_second_PC = np.zeros(n)
#         else:
#             transp_matrix_2_for_second_PC = [row[1] for row in data['d_star_jk']]
#         Ct_ij_2 = [row[1] for row in data['Ct_ij']]
#         Cr_jk_2 = [row[1] for row in data['Cr_jk_transposed']]
#         index_of_working_distr_node_2 = 1
#         demand_nodes_related_to_PC_1 = [i for i, row in enumerate(data['d_star_jk']) if row[0] != 0 and row[1] == 0]
#         demand_nodes_related_to_PC_2 = [i for i, row in enumerate(data['d_star_jk']) if row[1] != 0 and row[0] == 0]
#         demand_nodes_related_to_both = [i for i, row in enumerate(data['d_star_jk']) if row[1] != 0 and row[0] != 0]
        
#         #сценарий 1.1: поровну всем: 
#         total_available = sum(transp_matrix_1_for_first_PC + transp_matrix_1_for_second_PC)
#         a = int(sum(transp_matrix_1_for_first_PC + transp_matrix_1_for_second_PC) / n)
#         if all(x > a for x in data['d_k']):
#             remainder = sum(transp_matrix_1_for_first_PC + transp_matrix_1_for_second_PC) - a * n
#             add_q = np.zeros(n)
#             h = np.zeros(n)
#             for k in range(n):
#                 h[k] = data['s_k'][k] - (data['Cs_k'] + data['Cr_jk'])[k]
#             sorted_indices = sorted(range(n), key=lambda i: (-h[i]))
#             for k in sorted_indices:
#                 if data['d_k'][k] - a >= 0 and remainder != 0:
#                     if data['d_k'][k] - a > remainder:
#                         add_q[k] = remainder
#                         remainder = 0
#                     else:
#                         add_q[k] = data['d_k'][k] - a
#                         remainder -= (data['d_k'][k] - a)
#             a = add_q + a
#             q_without_broken_prod_node = q_without_broken_prod_node[1:]
#             if sum(transp_matrix_1_for_first_PC) == 0:
#                 cost_equall = a * data['Cr_jk'][1]
#                 print('сценарий 1.1:', sum(cost_equall))
#             elif sum(transp_matrix_1_for_second_PC) == 0:
#                 cost_equall = a * data['Cr_jk'][0]
#                 print('сценарий 1.1:', sum(cost_equall))
#             else:
#                 cost_equall, matrix_between_distr_and_demand_1 = optimal_transportation( a, q_without_broken_prod_node, data['Cr_jk_transposed'])
#                 print('сценарий 1.1:', cost_equall)
#         else:
#             print('не можем использовать сценарий 1.1:')
        


        
        # сценарий 1.1: поровну всем:
        # if sum(transp_matrix_1_for_first_PC) == 0:
        #     profit2, dist2 = scenario_2(transp_matrix_1_for_second_PC, transp_matrix_stop,
        #                       data, Ct_ij_2, Cr_jk_2, index_of_working_distr_node_2, n)
        #     if profit2 is not None:
        #         print('сценарий 1.1:', profit2)
        #     else:
        #         print('не можем использовать сценарий 1.1:')
        # elif sum(transp_matrix_1_for_second_PC) == 0:
        #     profit2, dist2 = scenario_2(transp_matrix_1_for_first_PC, transp_matrix_stop,
        #                       data, Ct_ij_1, Cr_jk_1, index_of_working_distr_node_1, n)
        #     if profit2 is not None:
        #         print('сценарий 1.1:', profit2)
        #     else:
        #         print('не можем использовать сценарий 1.1:')
        # else:
        #     a = int(sum(transp_matrix_1_for_first_PC + transp_matrix_1_for_second_PC) / n)
        #     if all(x > a for x in data['d_k']):
        #         remainder = sum(transp_matrix_1_for_first_PC + transp_matrix_1_for_second_PC) - a * n
        #         add_q = np.zeros(n)
        #         h = np.zeros(n)
        #         for k in range(n):
        #             h[k] = data['s_k'][k] - (data['Cs_k'] + data['Cr_jk'])[k]
        #         sorted_indices = sorted(range(n), key=lambda i: (-h[i]))
        #         for k in sorted_indices:
        #             if data['d_k'][k] - a >= 0 and remainder != 0:
        #                 if data['d_k'][k] - a > remainder:
        #                     add_q[k] = remainder
        #                     remainder = 0
        #                 else:
        #                     add_q[k] = data['d_k'][k] - a
        #                     remainder -= (data['d_k'][k] - a)
        #         a = add_q + a
        #         q_without_broken_prod_node = q_without_broken_prod_node[1:]
        #         print(a)
        #     else: 
        #         print('не можем использовать сценарий 1.1:')
            
        #     cost_equall, matrix_between_distr_and_demand_1 = optimal_transportation(a, q_without_broken_prod_node, data['Cr_jk_transposed'])
            # print('сценарий 1.1:', cost_equall, matrix_between_distr_and_demand_1)
            # total_cost_1_1 = np.round(sum(x * y for x, y in zip(np.sum(q_without_broken_prod_node, axis=0), np.array(data['Cp_i']))) + 
            #                        (np.array(data['Cd_j']) * np.sum(q_without_broken_prod_node, axis=0)) + 
            #                         sum(np.array(data['Cs_k']) * a) + 
            #                         sum(x * y for x, y in zip(np.array(data['Ct_ij']), q_without_broken_prod_node)) + 
            #                         sum(x * y for x, y in zip(a, data['Cr_jk'])), decimals = 2)
            # total_value_1_1 = np.round(sum(a * (np.array(data['s_k']))), decimals=2)
            # total_profit_1_1 = np.round(total_value_1_1 - total_cost_1_1, decimals=2)
     

        # сценарий 1.2: весовые коэффициенты 
        # if sum(transp_matrix_1_for_first_PC) == 0: 
        #     profit3, dist3 = scenario_3(transp_matrix_1_for_second_PC, transp_matrix_stop,
        #                       data, Ct_ij_2, Cr_jk_2, index_of_working_distr_node_2, n)
        #     print('сценарий 1.2:', profit3)
        # elif sum(transp_matrix_2_for_second_PC) == 0:
        #     profit3, dist3 = scenario_3(transp_matrix_1_for_first_PC, transp_matrix_stop,
        #                       data, Ct_ij_1, Cr_jk_1, index_of_working_distr_node_1, n)
        #     print('сценарий 1.2:', profit3)
        # else:
        #     W = np.array(data['d_k']) / data['D']
        #     prop = W * sum(transp_matrix_1_for_first_PC + transp_matrix_1_for_second_PC)
        #     int_prop = [math.floor(p) for p in prop]
        #     remainder = sum(transp_matrix_1_for_first_PC + transp_matrix_1_for_second_PC) - sum(int_prop)
        #     add_q = np.zeros(n)
        #     h = np.zeros(n)
        #     for k in range(n):
        #         h[k] = data['s_k'][k] - (data['Cs_k'] + data['Cr_jk'])[k]
        #     sorted_indices = sorted(range(n), key=lambda i: (-h[i]))
        #     for k in sorted_indices:
        #         if data['d_k'][k] - int_prop[k] > 0 and remainder != 0:
        #             if data['d_k'][k] - int_prop[k] > remainder:
        #                 add_q[k] = remainder
        #                 remainder = 0
        #             else:
        #                 add_q[k] = data['d_k'][k] - int_prop[k]
        #                 remainder -= (data['d_k'][k] - int_prop[k])
        #     int_prop = add_q + int_prop
        #     print(int_prop)
        #     cost_equall, matrix_between_distr_and_demand_1 = optimal_transportation(int_prop, q_without_broken_prod_node, data['Cr_jk_transposed'])
        #     print('сценарий 1.2:', cost_equall)

        



       
            

            
                    

            

            
        

In [49]:
def production_node_disruption(m, n, index_of_broken_prod_node, data):
    
    # 1. используем количество продукции, которое производит (m - 1) производственный узел
    print("1. используем количество продукции, которое производит (m - 1) производственный узел")
    q_without_broken_prod_node_all = copy.deepcopy(data['q_ij'])
    q_without_broken_prod_node_all[index_of_broken_prod_node] = [0, 0]
    q_without_broken_prod_node = [sum(x) for x in zip(*q_without_broken_prod_node_all)]
    transp_matrix_stop = np.zeros(m) # так как не замедляем процессы 

    index_of_uncovered_RC = [i for i, (a, b) in enumerate(zip([sum(col) for col in zip(*q_without_broken_prod_node_all)], np.array(data['mu']))) if a != b][0]
    uncovered_demands = np.array([row[index_of_uncovered_RC] for row in np.array(data['d_star_jk'])])
    covered_demands = np.array([row[1 - index_of_uncovered_RC] for row in np.array(data['d_star_jk'])])
    demand_nodes_related_to_uncovered_RC = [i for i, row in enumerate(np.array(data['d_star_jk'])) if row[index_of_uncovered_RC] != 0]
    available_in_uncovered_RC = [sum(col) for col in zip(*q_without_broken_prod_node_all)]
    transp_matrix_1_for_first_PC = [row[0] for row in q_without_broken_prod_node_all]
    if sum(transp_matrix_1_for_first_PC) == 0:
        transp_matrix_2_for_first_PC = np.zeros(n)
    else:
        transp_matrix_2_for_first_PC = [row[0] for row in data['d_star_jk']]
    Ct_ij_1 = [row[0] for row in data['Ct_ij']]
    Cr_jk_1 = [row[0] for row in data['Cr_jk_transposed']]
    index_of_working_distr_node_1 = 0 
    transp_matrix_1_for_second_PC = [row[1] for row in q_without_broken_prod_node_all]
    if sum(transp_matrix_1_for_second_PC) == 0:
        transp_matrix_2_for_second_PC = np.zeros(n)
    else:
        transp_matrix_2_for_second_PC = [row[1] for row in data['d_star_jk']]
    Ct_ij_2 = [row[1] for row in data['Ct_ij']]
    Cr_jk_2 = [row[1] for row in data['Cr_jk_transposed']]
    index_of_working_distr_node_2 = 1
    demand_nodes_related_to_PC_1 = [i for i, row in enumerate(data['d_star_jk']) if row[0] != 0 and row[1] == 0]
    demand_nodes_related_to_PC_2 = [i for i, row in enumerate(data['d_star_jk']) if row[1] != 0 and row[0] == 0]
    demand_nodes_related_to_both = [i for i, row in enumerate(data['d_star_jk']) if row[1] != 0 and row[0] != 0]
    # сбой производства, которое поствляет в оба РЦ
    if all(data['q_ij'][index_of_broken_prod_node]) !=0:
              
        #сценарий 1.1: поровну всем: 
        total_available = sum(transp_matrix_1_for_first_PC + transp_matrix_1_for_second_PC)
        a = int(sum(transp_matrix_1_for_first_PC + transp_matrix_1_for_second_PC) / n)
        if all(x > a for x in data['d_k']):
            remainder = sum(transp_matrix_1_for_first_PC + transp_matrix_1_for_second_PC) - a * n
            add_q = np.zeros(n)
            h = np.zeros(n)
            for k in range(n):
                h[k] = data['s_k'][k] - (data['Cs_k'] + data['Cr_jk'])[k]
            sorted_indices = sorted(range(n), key=lambda i: (-h[i]))
            for k in sorted_indices:
                if data['d_k'][k] - a >= 0 and remainder != 0:
                    if data['d_k'][k] - a > remainder:
                        add_q[k] = remainder
                        remainder = 0
                    else:
                        add_q[k] = data['d_k'][k] - a
                        remainder -= (data['d_k'][k] - a)
            a = add_q + a
            if sum(transp_matrix_1_for_first_PC) == 0:
                cost_equall = sum(a * data['Cr_jk'][1])
            elif sum(transp_matrix_1_for_second_PC) == 0:
                cost_equall = sum(a * data['Cr_jk'][0])
            else:
                cost_equall, matrix_between_distr_and_demand_1 = optimal_transportation( a, q_without_broken_prod_node, data['Cr_jk_transposed'])

            total_cost_equall = np.round (sum(sum(x * y for x, y in zip(np.array(q_without_broken_prod_node_all), np.array(data['Cp_i'])))) + 
                                 sum(np.array(data['Cd_j']) * q_without_broken_prod_node) + 
                                 sum(a * np.array(data['Cs_k'])) + 
                                 sum(sum(x * y for x, y in zip(np.array(data['Ct_ij']), np.array(q_without_broken_prod_node_all)))) + cost_equall, decimals = 2)
   
            total_value_equall = np.round(sum(a * np.array(data['s_k'])), decimals = 2)
            total_profit_equall = np.round(total_value_equall - total_cost_equall, decimals = 2)
            print('сценарий 1.1:', total_profit_equall) 
        else:
            print('не можем использовать сценарий 1.1:')


        # сценарий 1.2: весовые коэффициенты
        W = np.array(data['d_k']) / data['D']
        prop = W * sum(transp_matrix_1_for_first_PC + transp_matrix_1_for_second_PC)
        int_prop = [math.floor(p) for p in prop]
        remainder_prop = sum(transp_matrix_1_for_first_PC + transp_matrix_1_for_second_PC) - sum(int_prop)
        add_q = np.zeros(n)
        h = np.zeros(n)
        for k in range(n):
            h[k] = data['s_k'][k] - (data['Cs_k'] + data['Cr_jk'])[k]
        sorted_indices = sorted(range(n), key=lambda i: (-h[i]))
        for k in sorted_indices:
            if data['d_k'][k] - int_prop[k] > 0 and remainder_prop != 0:
                if data['d_k'][k] - int_prop[k] > remainder_prop:
                    add_q[k] = remainder_prop
                    remainder_prop = 0
                else:
                    add_q[k] = data['d_k'][k] - int_prop[k]
                    remainder_prop -= (data['d_k'][k] - int_prop[k])
        int_prop = add_q + int_prop
        if sum(transp_matrix_1_for_first_PC) == 0:
            cost_weight = sum(int_prop * data['Cr_jk'][1])
            # print('сценарий 1.2:', sum(cost_weight))
        elif sum(transp_matrix_1_for_second_PC) == 0:
            cost_weight = sum(int_prop * data['Cr_jk'][0])
            # print('сценарий 1.2:', sum(cost_weight))
        else:
            cost_weight, matrix_between_distr_and_demand_1 = optimal_transportation(int_prop, q_without_broken_prod_node, data['Cr_jk_transposed'])
            # print('сценарий 1.2:', cost_weight)
        total_cost_weight = np.round (sum(sum(x * y for x, y in zip(np.array(q_without_broken_prod_node_all), np.array(data['Cp_i'])))) + 
                                 sum(np.array(data['Cd_j']) * q_without_broken_prod_node) + 
                                 sum(int_prop * np.array(data['Cs_k'])) + 
                                 sum(sum(x * y for x, y in zip(np.array(data['Ct_ij']), np.array(q_without_broken_prod_node_all)))) + cost_weight, decimals = 2)

        total_value_weight = np.round(sum(int_prop * np.array(data['s_k'])), decimals = 2)
        total_profit_weight = np.round(total_value_weight - total_cost_weight, decimals = 2)
        print('сценарий 1.2:', total_profit_weight)
        
        # сценарий 1.3: по cортировка узлов спроса по наибольшей выручке для максимизации прибыли
        h = np.zeros(n)
        filling = np.zeros(n)
        remainder_h = sum(transp_matrix_1_for_first_PC + transp_matrix_1_for_second_PC)
    
        for k in range(n):
            h[k] = data['s_k'][k] - (data['Cs_k'] + data['Cr_jk'])[k]
        sorted_indices = sorted(range(n), key=lambda k: -h[k])
        for k in sorted_indices:
            if data['d_k'][k] > remainder_h and remainder_h != 0:
                filling[k] = remainder_h
                remainder_h = 0
            elif data['d_k'][k] < remainder_h and remainder_h != 0:
                filling[k] = data['d_k'][k]
                remainder_h -= data['d_k'][k]
        if sum(transp_matrix_1_for_first_PC) == 0:
            cost_h = sum(filling * data['Cr_jk'][1])
            # print('сценарий 1.3:', sum(cost_h))
        elif sum(transp_matrix_1_for_second_PC) == 0:
            cost_h = sum(filling * data['Cr_jk'][0])
            # print('сценарий 1.3:', sum(cost_h))
        else:
            cost_h, matrix_between_distr_and_demand_1 = optimal_transportation(filling, q_without_broken_prod_node, data['Cr_jk_transposed'])
            # print('сценарий 1.3:', cost_h)
        total_cost_h = np.round (sum(sum(x * y for x, y in zip(np.array(q_without_broken_prod_node_all), np.array(data['Cp_i'])))) + 
                                 sum(np.array(data['Cd_j']) * q_without_broken_prod_node) + 
                                 sum(filling * np.array(data['Cs_k'])) + 
                                 sum(sum(x * y for x, y in zip(np.array(data['Ct_ij']), np.array(q_without_broken_prod_node_all)))) + cost_h, decimals = 2)
   
        total_value_h = np.round(sum(filling * np.array(data['s_k'])), decimals = 2)
        total_profit_h = np.round(total_value_h - total_cost_h, decimals = 2)
        print('сценарий 1.3:', total_profit_h)

        #сценарий 4: сортировка про выручке за единицу в узле
        filling_2 = np.zeros(n)
        remainder_2 = sum(transp_matrix_1_for_first_PC + transp_matrix_1_for_second_PC)
        sorted_indices_profit = sorted(range(n), key=lambda i: -data['profit_per_unit'][i])
    
        for k in sorted_indices_profit:
            if data['d_k'][k] > remainder_2 and remainder_2 != 0:
                filling_2[k] = remainder_2
                remainder_2 = 0
            elif data['d_k'][k] < remainder_2 and remainder_2 != 0:
                filling_2[k] = data['d_k'][k]
                remainder_2 -= data['d_k'][k]
        if sum(transp_matrix_1_for_first_PC) == 0:
            cost_profit = sum(filling_2 * data['Cr_jk'][1])
            # print('сценарий 1.4:', sum(cost_profit))
        elif sum(transp_matrix_1_for_second_PC) == 0:
            cost_profit = sum(filling_2 * data['Cr_jk'][0])
            # print('сценарий 1.4:', sum(cost_profit))
        else:
            cost_profit, matrix_between_distr_and_demand_1 = optimal_transportation(filling_2, q_without_broken_prod_node, data['Cr_jk_transposed'])
            # print('сценарий 1.4:', cost_profit)
        total_cost_profit = np.round (sum(sum(x * y for x, y in zip(np.array(q_without_broken_prod_node_all), np.array(data['Cp_i'])))) + 
                                 sum(np.array(data['Cd_j']) * q_without_broken_prod_node) + 
                                 sum(filling_2 * np.array(data['Cs_k'])) + 
                                 sum(sum(x * y for x, y in zip(np.array(data['Ct_ij']), np.array(q_without_broken_prod_node_all)))) + cost_profit, decimals = 2)
   
        total_value_profit = np.round(sum(filling_2 * np.array(data['s_k'])), decimals = 2)
        total_profit_profit = np.round(total_value_profit - total_cost_profit, decimals = 2)
        print('сценарий 1.4:', total_profit_profit)

    # сбой производства, которое поставляло только в один рц
    else:
        q_without_broken_prod_node = [sum(x) for x in zip(*q_without_broken_prod_node_all)]

        # сценарий 1.1: поровну всем 
        a = int([sum(col) for col in zip(*q_without_broken_prod_node_all)][index_of_uncovered_RC] / len(demand_nodes_related_to_uncovered_RC))
        a_array = [a] * n
        for k in range(n):
             if uncovered_demands[k] == 0:
                a_array[k] = 0
        if all(x >= y for x, y in zip(uncovered_demands, a_array)):
            remainder = ([sum(col) for col in zip(*q_without_broken_prod_node_all)][index_of_uncovered_RC]) - a * len(demand_nodes_related_to_uncovered_RC)
            add_q = np.zeros(n)
            h = np.zeros(n)
            for k in range(n):
                h[k] = data['s_k'][k] - (data['Cs_k'] + data['Cr_jk'])[k]
            sorted_indices = sorted(range(n), key=lambda i: (-h[i]))
            for k in sorted_indices:
                if data['d_k'][k] - a >= 0 and remainder != 0 and a_array[k]!=0:
                    if data['d_k'][k] - a > remainder:
                        add_q[k] = remainder
                        remainder = 0
                    else:
                        add_q[k] = data['d_k'][k] - a
                        remainder -= (data['d_k'][k] - a)
            a = add_q + a_array
            if index_of_uncovered_RC == 0:
                d_star = [a, covered_demands]
            else:
                d_star = [covered_demands, a]

            total_cost_equall = np.round (sum(sum(x * y for x, y in zip(np.array(q_without_broken_prod_node_all), np.array(data['Cp_i'])))) + 
                                 sum(np.array(data['Cd_j']) * q_without_broken_prod_node) + 
                                 sum((a + covered_demands) * np.array(data['Cs_k'])) + 
                                 sum(sum(x * y for x, y in zip(np.array(data['Ct_ij']), np.array(q_without_broken_prod_node_all)))) + 
                                 sum(sum(x * y for x, y in zip(np.array(data['Cr_jk']), d_star))), decimals = 2)
            total_value_equall = np.round(sum((a + covered_demands) * np.array(data['s_k'])), decimals = 2)
            total_profit_equall = np.round(total_value_equall - total_cost_equall, decimals = 2)
            print('сценарий 1.1:', total_profit_equall) 
        else:
            print('не можем использовать сценарий 1.1:')
        

        # сценарий 1.2: весовые коэффициенты
        W = np.array(uncovered_demands/ data['mu'][index_of_uncovered_RC])
        prop = W * [sum(col) for col in zip(*q_without_broken_prod_node_all)][index_of_uncovered_RC]
        int_prop = [math.floor(p) for p in prop]
        remainder_prop = [sum(col) for col in zip(*q_without_broken_prod_node_all)][index_of_uncovered_RC] - sum(int_prop)
        add_q = np.zeros(n)
        h = np.zeros(n)
        for k in range(n):
            h[k] = data['s_k'][k] - (data['Cs_k'] + data['Cr_jk'])[k]
        sorted_indices = sorted(range(n), key=lambda i: (-h[i]))
        for k in sorted_indices:
            if data['d_k'][k] - int_prop[k] > 0 and remainder_prop != 0 and int_prop[k]!=0:
                if data['d_k'][k] - int_prop[k] > remainder_prop:
                    add_q[k] = remainder_prop
                    remainder_prop = 0
                else:
                    add_q[k] = data['d_k'][k] - int_prop[k]
                    remainder_prop -= (data['d_k'][k] - int_prop[k])
        int_prop = add_q + int_prop
        if index_of_uncovered_RC == 0:
            d_star_prop = [int_prop, covered_demands]
        else:
            d_star_prop = [covered_demands, int_prop]

        total_cost_prop = np.round (sum(sum(x * y for x, y in zip(np.array(q_without_broken_prod_node_all), np.array(data['Cp_i'])))) + 
                                 sum(np.array(data['Cd_j']) * q_without_broken_prod_node) + 
                                 sum((int_prop + covered_demands) * np.array(data['Cs_k'])) + 
                                 sum(sum(x * y for x, y in zip(np.array(data['Ct_ij']), np.array(q_without_broken_prod_node_all)))) + 
                                 sum(sum(x * y for x, y in zip(np.array(data['Cr_jk']), d_star_prop))), decimals = 2)
        total_value_prop = np.round(sum((int_prop  + covered_demands) * np.array(data['s_k'])), decimals = 2)
        total_profit_prop = np.round(total_value_prop - total_cost_prop, decimals = 2)
        print('сценарий 1.2:', total_profit_prop) 
        
        # сценарий 4: cортировка узлов спроса по наибольшей выручке для максимизации прибыли
        filling = np.zeros(n)
        remainder_h = [sum(col) for col in zip(*q_without_broken_prod_node_all)][index_of_uncovered_RC]
        h_new = np.zeros(n)
        for k in range(n):
            if uncovered_demands[k] != 0:
                h_new[k] = data['s_k'][k] - (data['Cs_k'] + data['Cr_jk'])[k]
        nonzero_indices = [i for i in range(n) if uncovered_demands[i] != 0]
        sorted_indices_h = sorted(nonzero_indices, key=lambda i: -h_new[i])
        for k in sorted_indices_h:
            if data['d_k'][k] > remainder_h and remainder_h != 0:
                filling[k] = remainder_h
                remainder_h = 0
            elif data['d_k'][k] < remainder_h and remainder_h != 0:
                filling[k] = data['d_k'][k]
                remainder_h -= data['d_k'][k]
        if index_of_uncovered_RC == 0:
            d_star_h = [filling, covered_demands]
        else:
            d_star_h = [covered_demands, filling]
        total_cost_h = np.round(sum(sum(x * y for x, y in zip(np.array(q_without_broken_prod_node_all), np.array(data['Cp_i'])))) + 
                                 sum(np.array(data['Cd_j']) * q_without_broken_prod_node) + 
                                 sum((filling + covered_demands) * np.array(data['Cs_k'])) + 
                                 sum(sum(x * y for x, y in zip(np.array(data['Ct_ij']), np.array(q_without_broken_prod_node_all)))) + 
                                 sum(sum(x * y for x, y in zip(np.array(data['Cr_jk']), d_star_h))), decimals = 2)
        total_value_h = np.round(sum((filling  + covered_demands) * np.array(data['s_k'])), decimals = 2)
        total_profit_h = np.round(total_value_h - total_cost_h, decimals=2)
        print('сценарий 1.3:', total_profit_h) 


        #сценарий 5: сортировка про выручке за единицу в узле
        filling_prof = np.zeros(n)
        remainder_prof = [sum(col) for col in zip(*q_without_broken_prod_node_all)][index_of_uncovered_RC]
        nonzero_indices_prof = [i for i in range(n) if uncovered_demands[i] != 0]
        sorted_indices_prof = sorted(nonzero_indices_prof, key=lambda i: -data['profit_per_unit'][i])
    
        for k in sorted_indices_prof:
            if data['d_k'][k] > remainder_prof and remainder_prof != 0:
                filling_prof[k] = remainder_prof
                remainder_prof = 0
            elif data['d_k'][k] < remainder_prof and remainder_prof != 0:
                filling_prof[k] = data['d_k'][k]
                remainder_prof -= data['d_k'][k]
        if index_of_uncovered_RC == 0:
            d_star_prof = [filling_prof, covered_demands]
        else:
            d_star_prof = [covered_demands, filling_prof]
        total_cost_prof = np.round(sum(sum(x * y for x, y in zip(np.array(q_without_broken_prod_node_all), np.array(data['Cp_i'])))) + 
                                 sum(np.array(data['Cd_j']) * q_without_broken_prod_node) + 
                                 sum((filling_prof + covered_demands) * np.array(data['Cs_k'])) + 
                                 sum(sum(x * y for x, y in zip(np.array(data['Ct_ij']), np.array(q_without_broken_prod_node_all)))) + 
                                 sum(sum(x * y for x, y in zip(np.array(data['Cr_jk']), d_star_prof))), decimals = 2)
        total_value_prof = np.round(sum((filling_prof  + covered_demands) * np.array(data['s_k'])), decimals = 2)
        total_profit_prof = np.round(total_value_prof - total_cost_prof, decimals=2)
        print('сценарий 1.4:', total_profit_prof) 

    # повышаем мощности производств
    print('2. повышение мощностей производств')
    u = np.zeros(m)
    Ct_ij_for_uncovered_RC  = [row[index_of_uncovered_RC] for row in data['Ct_ij']]
    
    for i in range(m):
        u[i] = data['Cp_i'][i] + data['Cp_i_add'][i] + Ct_ij_for_uncovered_RC[i]
    nonzero_indices_add = [i for i in range(m) if i != 0]
    sorted_indices_add = sorted(nonzero_indices_add, key=lambda i: u[i])
    deficit = data['D'] - sum(available_in_uncovered_RC)
    increased_nu = np.zeros(m)
    for i in sorted_indices_add:
        increased_nu[i] =  data['nu_add'][i]
    distribution_of_increased_production = np.zeros((m, 2))
    remainder_add = np.array(data['mu_add']) 
   
    for i in range(m):
        # определяем предпочтительный центр для направления доп продукции
        if data['Ct_ij'][i][0] + data['Cd_j'][0] > data['Ct_ij'][i][1] + data['Cd_j'][1]:
            preferred_j = 1
            alternative_j = 0
        else:
            preferred_j = 0
            alternative_j = 1
        
        # распределяем в предпочтительный 
        if remainder_add[preferred_j] > 0:
            if remainder_add[preferred_j] >= increased_nu[i]:
                distribution_of_increased_production[i][preferred_j] = increased_nu[i]
                remainder_add[preferred_j] -= increased_nu[i]
            else:
                
                distributed = remainder_add[preferred_j]
                distribution_of_increased_production[i][preferred_j] = distributed
                remainder_add[preferred_j] = 0
                
                # остаток пытаемся распределить в другой рц
                remaining = increased_nu[i] - distributed
                if remainder_add[alternative_j] >= remaining:
                    distribution_of_increased_production[i][alternative_j] += remaining
                    remainder_add[alternative_j] -= remaining
                else:
                    distribution_of_increased_production[i][alternative_j] += remainder_add[alternative_j]
                    remainder_add[alternative_j] = 0
        else:
            # если предпочтительный центр уже заполнен полностью
            if remainder_add[alternative_j] >= increased_nu[i]:
                distribution_of_increased_production[i][alternative_j] = increased_nu[i]
                remainder_add[alternative_j] -= increased_nu[i]
            else:
                distribution_of_increased_production[i][alternative_j] = remainder_add[alternative_j]
                remainder_add[alternative_j] = 0
    
    distribution_of_increased_production = distribution_of_increased_production + q_without_broken_prod_node_all
    distribution_of_increased_production_not_all = [sum(x) for x in zip(*distribution_of_increased_production)]
    q_without_broken_prod_node_all_add = copy.deepcopy(data['nu_add'])
    q_without_broken_prod_node_all_add[index_of_broken_prod_node] = 0 
    
    q = [sum(x) for x in zip(*distribution_of_increased_production)]
    add_cost_in_PC = np.zeros(2)
    for j in range(2):
        if q[j] > data['mu'][j]:
            add_cost_in_PC[j] = (q[j] - data['mu'][j]) * data['Cd_j_add'][j]
   

    a = int(np.sum(distribution_of_increased_production) / n)
    if all(x > a for x in data['d_k']):
        remainder = np.sum(distribution_of_increased_production) - a * n
        add_q = np.zeros(n)
        h = np.zeros(n)
        for k in range(n):
            h[k] = data['s_k'][k] - (data['Cs_k'] + data['Cr_jk'])[k]
        sorted_indices = sorted(range(n), key=lambda i: (-h[i]))
        for k in sorted_indices:
            if data['d_k'][k] - a >= 0 and remainder != 0:
                if data['d_k'][k] - a > remainder:
                    add_q[k] = remainder
                    remainder = 0
                else:
                    add_q[k] = data['d_k'][k] - a
                    remainder -= (data['d_k'][k] - a)
        a = add_q + a
        if q[0] == 0:
            cost_equall = sum(a * data['Cr_jk'][1])
        elif q[1] == 0:
            cost_equall = sum(a * data['Cr_jk'][0])
        else:
            cost_equall, matrix_between_distr_and_demand_1 = optimal_transportation( a, q, data['Cr_jk_transposed'])
        total_cost_equall = np.round(sum(sum(x * y for x, y in zip(np.array(q_without_broken_prod_node_all), np.array(data['Cp_i'])))) + 
                             sum(x * y for x, y in zip(np.array(q_without_broken_prod_node_all_add), np.array(data['Cp_i_add']))) +
                             sum(np.array(data['Cd_j']) * distribution_of_increased_production_not_all) + np.sum(add_cost_in_PC) +
                             sum(a * np.array(data['Cs_k'])) + 
                             sum(sum(x * y for x, y in zip(np.array(data['Ct_ij']), np.array(distribution_of_increased_production)))) + cost_equall, decimals = 2)
        total_value_equall = np.round(sum(a * np.array(data['s_k'])), decimals = 2)
        total_profit_equall = np.round(total_value_equall - total_cost_equall, decimals = 2)
        print('сценарий 2.1:', total_profit_equall) 
    else:
        print('не можем использовать сценарий 2.1:')

    # сценарий 1.2: весовые коэффициенты
    W = np.array(data['d_k']) / data['D']
    prop = W * np.sum(distribution_of_increased_production)
    int_prop = [math.floor(p) for p in prop]
    remainder_prop = np.sum(distribution_of_increased_production) - sum(int_prop)
    add_q = np.zeros(n)
    h = np.zeros(n)
    for k in range(n):
        h[k] = data['s_k'][k] - (data['Cs_k'] + data['Cr_jk'])[k]
    sorted_indices = sorted(range(n), key=lambda i: (-h[i]))
    for k in sorted_indices:
        if data['d_k'][k] - int_prop[k] > 0 and remainder_prop != 0:
            if data['d_k'][k] - int_prop[k] > remainder_prop:
                add_q[k] = remainder_prop
                remainder_prop = 0
            else:
                add_q[k] = data['d_k'][k] - int_prop[k]
                remainder_prop -= (data['d_k'][k] - int_prop[k])
    int_prop = add_q + int_prop
    if q[0] == 0:
        cost_weight = sum(int_prop * data['Cr_jk'][1])
        # print('сценарий 1.2:', sum(cost_weight))
    elif q[1] == 0:
        cost_weight = sum(int_prop * data['Cr_jk'][0])
        # print('сценарий 1.2:', sum(cost_weight))
    else:
        cost_weight, matrix_between_distr_and_demand_1 = optimal_transportation(int_prop, q, data['Cr_jk_transposed'])
        # print('сценарий 1.2:', cost_weight)
    total_cost_weight = np.round(sum(sum(x * y for x, y in zip(np.array(q_without_broken_prod_node_all), np.array(data['Cp_i'])))) + 
                             sum(x * y for x, y in zip(np.array(q_without_broken_prod_node_all_add), np.array(data['Cp_i_add']))) +
                             sum(np.array(data['Cd_j']) * distribution_of_increased_production_not_all) + np.sum(add_cost_in_PC) +
                             sum(int_prop * np.array(data['Cs_k'])) + 
                             sum(sum(x * y for x, y in zip(np.array(data['Ct_ij']), np.array(distribution_of_increased_production)))) + cost_weight, decimals = 2)
    total_value_weight = np.round(sum(int_prop * np.array(data['s_k'])), decimals = 2)
    total_profit_weight = np.round(total_value_weight - total_cost_weight, decimals = 2)
    print('сценарий 2.2:', total_profit_weight)
   
    # сценарий 2.3: по cортировка узлов спроса по наибольшей выручке для максимизации прибыли
    h = np.zeros(n)
    filling = np.zeros(n)
    remainder_h = np.sum(distribution_of_increased_production)
    for k in range(n):
        h[k] = data['s_k'][k] - (data['Cs_k'] + data['Cr_jk'])[k]
    sorted_indices = sorted(range(n), key=lambda k: -h[k])
    for k in sorted_indices:
        if data['d_k'][k] > remainder_h and remainder_h != 0:
            filling[k] = remainder_h
            remainder_h = 0
        elif data['d_k'][k] < remainder_h and remainder_h != 0:
            filling[k] = data['d_k'][k]
            remainder_h -= data['d_k'][k]
    if q[0] == 0:
        cost_h = sum(filling * data['Cr_jk'][1])
        # print('сценарий 1.3:', sum(cost_h))
    elif q[1] == 0:
        cost_h = sum(filling * data['Cr_jk'][0])
        # print('сценарий 1.3:', sum(cost_h))
    else:
        cost_h, matrix_between_distr_and_demand_1 = optimal_transportation(filling, q, data['Cr_jk_transposed'])
        # print('сценарий 1.3:', cost_h)
    total_cost_h = np.round(sum(sum(x * y for x, y in zip(np.array(q_without_broken_prod_node_all), np.array(data['Cp_i'])))) + 
                             sum(x * y for x, y in zip(np.array(q_without_broken_prod_node_all_add), np.array(data['Cp_i_add']))) +
                             sum(np.array(data['Cd_j']) * distribution_of_increased_production_not_all) + np.sum(add_cost_in_PC) +
                             sum(filling * np.array(data['Cs_k'])) + 
                             sum(sum(x * y for x, y in zip(np.array(data['Ct_ij']), np.array(distribution_of_increased_production)))) + cost_h, decimals = 2)
    
    total_value_h = np.round(sum(filling * np.array(data['s_k'])), decimals = 2)
    total_profit_h = np.round(total_value_h - total_cost_h, decimals = 2)
    print('сценарий 2.3:', total_profit_h)
    #сценарий 4: сортировка про выручке за единицу в узле
    filling_2 = np.zeros(n)
    remainder_2 = np.sum(distribution_of_increased_production)
    sorted_indices_profit = sorted(range(n), key=lambda i: -data['profit_per_unit'][i])

    for k in sorted_indices_profit:
        if data['d_k'][k] > remainder_2 and remainder_2 != 0:
            filling_2[k] = remainder_2
            remainder_2 = 0
        elif data['d_k'][k] < remainder_2 and remainder_2 != 0:
            filling_2[k] = data['d_k'][k]
            remainder_2 -= data['d_k'][k]
    if q[0] == 0:
        cost_profit = sum(filling_2 * data['Cr_jk'][1])
        # print('сценарий 1.4:', sum(cost_profit))
    elif q[1] == 0:
        cost_profit = sum(filling_2 * data['Cr_jk'][0])
        # print('сценарий 1.4:', sum(cost_profit))
    else:
        cost_profit, matrix_between_distr_and_demand_1 = optimal_transportation(filling_2, q, data['Cr_jk_transposed'])
        # print('сценарий 1.4:', cost_profit)
    total_cost_profit = np.round(sum(sum(x * y for x, y in zip(np.array(q_without_broken_prod_node_all), np.array(data['Cp_i'])))) + 
                             sum(x * y for x, y in zip(np.array(q_without_broken_prod_node_all_add), np.array(data['Cp_i_add']))) +
                             sum(np.array(data['Cd_j']) * distribution_of_increased_production_not_all) + np.sum(add_cost_in_PC) +
                             sum(filling_2 * np.array(data['Cs_k'])) + 
                             sum(sum(x * y for x, y in zip(np.array(data['Ct_ij']), np.array(distribution_of_increased_production)))) + cost_profit, decimals = 2)
    
    total_value_profit = np.round(sum(filling_2 * np.array(data['s_k'])), decimals = 2)
    total_profit_profit = np.round(total_value_profit - total_cost_profit, decimals = 2)
    print('сценарий 2.4:', total_profit_profit)

  
    
    
        

In [50]:
PC = 0
d_3 = production_node_disruption(m, n, PC, all_gen_data)


1. используем количество продукции, которое производит (m - 1) производственный узел
сценарий 1.1: 11041.46
сценарий 1.2: 11253.98
сценарий 1.3: 11474.34
сценарий 1.4: 12218.81
2. повышение мощностей производств
сценарий 2.1: 11041.46
сценарий 2.2: 11253.98
сценарий 2.3: 11474.34
сценарий 2.4: 12218.81


In [51]:
# генерируем матожидание t в каждом производстенном и распределительном узле
LOWER_BOUND_k = 0.5
UPPER_BOUND_k = 1.9

LOWER_BOUND_lambda = 1
UPPER_BOUND_lambda = 50

LOWER_BOUND_number_of_samples = 20
UPPER_BOUND_number_of_samples = 100

def generate_weibull_params(size, LOWER_BOUND_k, UPPER_BOUND_k, 
                          LOWER_BOUND_lambda, UPPER_BOUND_lambda,
                          LOWER_BOUND_number_of_samples, UPPER_BOUND_number_of_samples):
    
    k = np.random.uniform(LOWER_BOUND_k, UPPER_BOUND_k, size=size)
    lambda_ = np.random.randint(LOWER_BOUND_lambda, UPPER_BOUND_lambda, size=size)
    number_of_samples = np.random.randint(LOWER_BOUND_number_of_samples, 
                                        UPPER_BOUND_number_of_samples, 
                                        size=size)
    
    
    t = [np.round(weibull_min.rvs(k[i], scale=lambda_[i], size=number_of_samples[i])).astype(int)
         for i in range(size)]
    
    E = np.round(lambda_ * gamma(1 + 1 / k)).astype(int)
    return k, lambda_, number_of_samples, t, E

# Генерация для prod
k_prod, lambda_prod, number_of_samples_prod, t_prod, E_prod_hour = generate_weibull_params(
    m, LOWER_BOUND_k, UPPER_BOUND_k, LOWER_BOUND_lambda, UPPER_BOUND_lambda,
    LOWER_BOUND_number_of_samples, UPPER_BOUND_number_of_samples)
print("E[T] в производственных узлах = ", E_prod_hour)

# Генерация для distr
k_distr, lambda_distr, number_of_samples_distr, t_distr, E_distr_hour = generate_weibull_params(
    2, LOWER_BOUND_k, UPPER_BOUND_k, LOWER_BOUND_lambda, UPPER_BOUND_lambda,
    LOWER_BOUND_number_of_samples, UPPER_BOUND_number_of_samples)
print("E[T] в распределеительных узлах = ", E_distr_hour)

E_prod = np.ceil(E_prod_hour / 12)
E_distr = np.ceil(E_distr_hour / 12)
print(E_prod, E_distr)

E[T] в производственных узлах =  [48  4 34]
E[T] в распределеительных узлах =  [32 32]
[4. 1. 3.] [3. 3.]


In [52]:
def stocks(m, n, all_gen_data, E_distr, E_prod):
    losses_from_deficit_demand_node = all_gen_data['profit_per_unit']
    losses_from_deficit_distr_node = np.zeros(2)
    losses_from_deficit_prod_node = np.zeros(m)
    sum_2 = np.zeros(n)
    for j in range(2):
        for k in range(n):
            sum_2[k] = losses_from_deficit_demand_node[k] * all_gen_data['d_star_jk'][k][j]
        losses_from_deficit_distr_node[j] = np.sum(sum_2)/all_gen_data['mu'][j]
    sum_1 = np.zeros(m)
    for i in range(m):
        for j in range(2):
            sum_1[j] = losses_from_deficit_distr_node[j] * all_gen_data['q_ij'][i][j]
        losses_from_deficit_prod_node[i] =  np.sum(sum_1)/all_gen_data['nu'][j]
    S_prod = np.zeros(m)
    S_distr = np.zeros(2)
    S_demand =  np.zeros(n)
    for i in range(m):
        if (all_gen_data['nu'][i] * E_prod[i] - (all_gen_data['Hp_i'][i] * all_gen_data['nu'][i] * E_prod[i])/losses_from_deficit_prod_node[i]) > 0:
            S_prod[i] = min(np.floor(all_gen_data['nu'][i] * E_prod[i] - (all_gen_data['Hp_i'][i] * all_gen_data['nu'][i] * E_prod[i])/losses_from_deficit_prod_node[i]), all_gen_data['alpha'][i])
    print(all_gen_data['alpha'], S_prod)

    for j in range(2):
        if (all_gen_data['mu'][j] * E_distr[j] - (all_gen_data['Hd_j'][j] * all_gen_data['mu'][j] * E_distr[j])/losses_from_deficit_distr_node[j]) > 0:
            S_distr[j] = min(np.floor(all_gen_data['mu'][j] * E_distr[j] - (all_gen_data['Hd_j'][j] * all_gen_data['mu'][j] * E_distr[j])/losses_from_deficit_distr_node[j]), all_gen_data['beta'][j])
    print(all_gen_data['beta'], S_distr)

    



kdovso = stocks(m, n, all_gen_data, E_distr, E_prod)

[170.0, 150.0, 61.0] [170.  40.  61.]
[70.0, 272.0] [70.  0.]
